<a href="https://colab.research.google.com/github/glombardo/Research/blob/main/AI_TFP_Gains_Guido_Lombardo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SETUP

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

Sentiment = 0 #-1,0,1 <- used to experiment with exogenous noise.
# behavioral and info parameters
overconfidence_list = [0.0, 0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20, 1.0, 1.5, 2.0, 3.0, 4.0, 6.0, 7.0, 10.0] #theta
info_asym_list = [0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.50, 1.75, 2.0]
#info_asym_list = np.arange(0, 100) #need a LOT of ram for this to run

#supply/capacity
l_sup = 3000
ai_sup = 3000

#global or fixed params
np.random.seed(42) #reproducibility
time_horizon = 40 #horizon
years = np.arange(1, time_horizon+1)
N = 3000 # number of tasks in the market
annual_ai_growth = 0.00 # how AI productivity improves each year + exp model later
WAGE = 1.0 # fix wage & rental_rate for partial equilibrium (could solve for them endogenously)
RENTAL_RATE = 1.0

#simulate data (use histogram to experiment with different task/labor/ai overlaps)
task_complexities = np.random.normal(loc=3.0, scale=0.3, size=N)
labor_productivities = np.random.normal(loc=3.0, scale=0.3, size=N) #np.clip(labor_productivities_raw, 0.1, 5.0)
ai_productivities = np.random.normal(loc=2.5, scale=0.3, size=N)


## Labor, AI, and Task Distributions

In [ ]:
#for histogram later
df_dists = pd.DataFrame({
    "task_complexities": task_complexities,
    "labor_productivities": labor_productivities,
    "ai_productivities": ai_productivities
})

# unpivot so we have "Distribution" and "Value" (the data)
df_melted = df_dists.melt(
    var_name="Distribution",
    value_name="Value"
)

# Create an overlapping histogram
fig_hist = px.histogram(
    df_melted,
    x="Value",
    color="Distribution",      # which distribution each value belongs to
    nbins=250,                  # number of histogram bins
    opacity=0.5,               # transparency so they overlap visibly
    barmode="overlay",         # overlay all in one chart
    histnorm="density"         # optional: show normalized densities instead of counts
)

fig_hist.update_layout(
    title="Overlapping Distributions: Task Complexities, Labor & AI Productivities",
    xaxis_title="Value",
    yaxis_title="Density"
)

## SIMLUATION

In [ ]:


#speed+quality trade-off
def phi_speed_quality(t, complexity):
    """
    Example saturation function:
    phi(t, complexity) = 1 - exp( -t / (0.1 + 0.9*complexity) )
    Larger t => higher output (diminishing returns).
    'complexity' influences how quickly output saturates.
    """
    return 1.0 - np.exp(- t / (0.1 + 0.9*complexity))


#Overconfidence + Info Asymmetry + Speed-Quality


    ##Assign each task i to (AI) or (Labor) and choose time t in [0,0.5,1,2,3]
    ##to maximize perceived profit = perceived_Q - cost.

    ##Overconfidence => manager perceives AI productivity = actual_AI * (1 + theta).
    ##Info Asymmetry => manager sees complexity_i + N(0, info_asym_level).

    ##This function returns an array Q_realized of shape (N,)
    ##representing the actual (true) output from each task.


def manager_choice_new_approach(
    complexities,
    labor_prod_array,
    ai_prod_array,
    overconfidence_level,
    info_asym_level,
    labor_supply,
    ai_capacity,
    year
    ):

    N = len(complexities)
    # "true" AI scale factor in year "year":
    ai_scale_true = np.exp(annual_ai_growth * (year - 1)) #log_growth = a + b * np.log(years)
    #ai_scale_true = annual_ai_growth*np.log(year-1)
    #ai_scale_true = 1.0 + annual_ai_growth*(year-1)
    # manager's perceived AI scale:
    ai_scale_perceived = ai_scale_true * (1.0 + overconfidence_level)

    # We'll do a small grid search for t in possible_times:
    possible_times = [0.0, 0.5, 1.0, 2.0, 3.0]

    # We'll keep track of final realized outputs
    Q_realized = np.zeros(N)

    # For capacity constraints, we'll track usage of labor vs AI,
    # but won't do a full iterative feasibility approach (we just sum up).
    total_labor_used = 0.0
    total_ai_used    = 0.0

    # Info asymmetry => manager sees complexity + noise
    noise = np.random.normal(Sentiment, info_asym_level, size=N) # Sentiment
    perceived_complexities = complexities + noise
    #perceived_complexities = np.clip(perceived_complexities, 0.1, 10.0)

    # For simplicity, set wage = 1, p_ai = 1.
    # (If you want partial eq. with WAGE, RENTAL_RATE, just plug them in as cost.)
    wage = 1.0
    p_ai = 1.0

    for i in range(N):
        best_perceived_profit = -1e9
        best_method = 0
        best_time = 0.0

        # manager picks method in {0: labor, 1: AI} to maximize perceived profit
        for method in [0,1]:
            for t_ in possible_times:
                if method == 0:  # labor
                    q_perceived = labor_prod_array[i]*phi_speed_quality(t_, perceived_complexities[i])
                    cost = wage*t_
                else:           # AI
                    q_perceived = labor_prod_array[i] * ai_scale_perceived * phi_speed_quality(t_, perceived_complexities[i])
                    # NOTE: above, you multiply labor_prod_array[i] again for everything or you might prefer
                    # ai_prod_array[i] for the baseline AI factor. It's up to your model's structure.
                    # If you want to use ai_prod_array, do:  ai_prod_array[i]*ai_scale_perceived*phi_speed_quality(...)
                    # We'll keep your snippet's style for demonstration:
                    cost = p_ai*t_

                profit = q_perceived - cost
                if profit > best_perceived_profit:
                    best_perceived_profit = profit
                    best_method = method
                    best_time = t_

        # Now we know the manager's choice
        if best_method == 0:
            # actual realized Q = labor_prod_array[i] * phi_speed_quality(t_, true_complexity)
            Q_real = labor_prod_array[i]*phi_speed_quality(best_time, complexities[i])
            total_labor_used += best_time
        else:
            # actual realized Q = (some baseline AI measure) * ai_scale_true * phi(...)
            # More consistent with a "true" AI factor usage might be:
            Q_real = ai_prod_array[i]*ai_scale_true * phi_speed_quality(best_time, complexities[i])
            total_ai_used += best_time

        Q_realized[i] = Q_real

    # Optionally check if total_labor_used <= labor_supply, total_ai_used <= ai_capacity
    # For demonstration, we skip the feasibility fix but you could iterate or re-solve if usage is too large.

    return Q_realized

def compute_tfp(Q_values):
    """
    Suppose TFP is simply the average realized output:
      TFP = (sum of Q_i) / N
    Or use a share-weighted approach if you prefer.
    """
    return np.sum(Q_values) / len(Q_values)

# ----------------------------------------------------------------------------
# 4) "REFINED ACEMOGLU APPROACH": Factor Cost Comparison
# ----------------------------------------------------------------------------

def acemoglu_approach_tfp_refined(
    complexities,
    labor_prod_array,
    ai_prod_array,
    wage,
    rental_rate,
    year,
    annual_ai_growth=0.02
    ):
    """
    A more 'Acemoglu-like' approach:
      1) True AI productivity each year = ai_prod_array[i] * (1 + annual_ai_growth*(year-1)).
      2) Compare unit cost of labor vs. AI for each task:
           cost_of_labor_i = wage / labor_prod_array[i]
           cost_of_ai_i    = rental_rate / ( ai_prod_array[i]*AI_growth_factor )
      3) Assign each task i to whichever factor yields lower unit cost.
      4) Realized Q for that task = factor's "true productivity"
         (no speed-quality dimension).
    """
    N = len(complexities)
    Q_acemoglu = np.zeros(N)

    #AI_growth_factor = 1.0 + annual_ai_growth*(year-1)
    AI_growth_factor = np.exp(annual_ai_growth * (year - 1))

    for i in range(N):
        # labor: unit cost = wage / labor_prod
        labor_unit_cost = wage / labor_prod_array[i]
        # AI: unit cost = rental_rate / [ai_prod_i * AI_growth_factor]
        ai_unit_cost    = rental_rate / (ai_prod_array[i] * AI_growth_factor)

        if ai_unit_cost < labor_unit_cost:
            # Assign to AI
            Q_acemoglu[i] = ai_prod_array[i] * AI_growth_factor
        else:
            # Assign to labor
            Q_acemoglu[i] = labor_prod_array[i]

    return Q_acemoglu

# ----------------------------------------------------------------------------
# 5) MAIN SIMULATION / RUN SCENARIOS
# ----------------------------------------------------------------------------

def run_scenarios(overconfidence_list, info_asym_list):
    """
    We'll store TFP paths for the 'new approach' under various (theta, alpha),
    as well as a single TFP path for the refined Acemoglu approach.
    """
    results_new = {}
    # We'll do a single run for each (theta, alpha)

    for oc in overconfidence_list:
        for ia in info_asym_list:
            key_name = f"OC={oc},IA={ia}"
            tfp_path = []
            for yr in years:
                # Use new approach
                Q_new = manager_choice_new_approach(
                    complexities       = task_complexities,
                    labor_prod_array   = labor_productivities,
                    ai_prod_array      = ai_productivities,
                    overconfidence_level = oc,
                    info_asym_level    = ia,
                    labor_supply       = l_sup,
                    ai_capacity        = ai_sup,
                    year               = yr
                )
                tfp_path.append(compute_tfp(Q_new))
            results_new[key_name] = np.array(tfp_path)

    # Also do one path for the refined Acemoglu approach
    tfp_acemoglu_path = []
    for yr in years:
        Q_ace = acemoglu_approach_tfp_refined(
            complexities      = task_complexities,
            labor_prod_array  = labor_productivities,
            ai_prod_array     = ai_productivities,
            wage              = WAGE,
            rental_rate       = RENTAL_RATE,
            year              = yr,
            annual_ai_growth  = annual_ai_growth
        )
        tfp_acemoglu_path.append(compute_tfp(Q_ace))

    tfp_acemoglu_path = np.array(tfp_acemoglu_path)
    return results_new, tfp_acemoglu_path


##test

In [ ]:
results_new, results_ace = run_scenarios(overconfidence_list, info_asym_list)

<ipython-input-8-d188c566ca7b>:9: RuntimeWarning:

overflow encountered in exp

<ipython-input-8-d188c566ca7b>:74: RuntimeWarning:

overflow encountered in scalar multiply

<ipython-input-8-d188c566ca7b>:77: RuntimeWarning:

overflow encountered in scalar multiply



In [ ]:
data_dict = {}
for key, arr in results_new.items():
    # key format: "OC=<val>,IA=<val>"
    oc_str, ia_str = key.split(',')
    oc_val = float(oc_str.split('=')[1])
    ia_val = float(ia_str.split('=')[1])
    avg_val = np.mean(arr)

    # Organize data by IA -> {OC: average}
    data_dict.setdefault(ia_val, {})
    data_dict[ia_val][oc_val] = avg_val

# Step 2: Sort IA values so we can assign them a color from a gradient
sorted_ia = sorted(data_dict.keys())
num_ia = len(sorted_ia)

# Choose a color scale. You can pick another from px.colors.sequential (e.g., Viridis, Plasma, etc.).
color_scale = px.colors.sequential.Blues

fig = go.Figure()

# Step 3: Create a trace for each IA with a unique color from the gradient
for i, ia_val in enumerate(sorted_ia):
    oc_dict = data_dict[ia_val]
    # Sort the OC values for a clean line in ascending order
    sorted_oc = sorted(oc_dict.keys())
    y_values = [oc_dict[oc] for oc in sorted_oc]

    # Map current IA index i to the color scale:
    # We'll pick a color by evenly stepping through the color_scale list.
    color_idx = int(i * (len(color_scale) - 1) / (num_ia - 1)) if num_ia > 1 else 0
    line_color = color_scale[color_idx]

    fig.add_trace(
        go.Scatter(
            x=sorted_oc,
            y=y_values,
            mode='lines+markers',
            name=f'IA={ia_val}',
            line=dict(color=line_color),
            marker=dict(color=line_color)
        )
    )

fig.update_layout(
    title='TFP Est. vs Over-Confidence',
    xaxis_title='Over-confidence (OC)',
    yaxis_title='TFP Est.',
    legend_title='Information Asymmetry (IA)'
)

fig.show()

## Prepare Data Viz

In [ ]:
results_new, results_ace = run_scenarios(overconfidence_list, info_asym_list)
fig = go.Figure()

# Add Acemoglu's line
fig.add_trace(
    go.Scatter(
        x = years,
        y = results_ace,
        mode = 'lines',
        name = 'Acemoglu',
        hovertemplate = (
            'Year=%{x}<br>'
            + 'TFP=%{y:.4f}<br>'
            + 'Approach=Acemoglu'
        )
    )
)

# Add lines for every (overconfidence, info_asym) pair
#for label_str, tfp_vals in results_new.items():
#    fig.add_trace(
#        go.Scatter(
#            x = years,
#            y = tfp_vals,
#            mode = 'lines',
#            name = label_str,  # e.g. "OC=0.3,IA=0.05"
#            hovertemplate = (
#                'TFP=%{y:.4f}<br>'
#                + label_str
#            )
#        )
#    )

def distance_from_center(label):
    oc_str, ia_str = label.split(',')
    oc = float(oc_str.split('=')[1])
    ia = float(ia_str.split('=')[1])
    return np.sqrt((oc - 1)**2 + (ia - 1)**2)

# Normalize distances
labels = list(results_new.keys())
distances = np.array([distance_from_center(label) for label in labels])
max_dist = distances.max() if distances.max() > 0 else 1
normalized = distances / max_dist

# Colormap: RdYlGn — green (close to (0,0)), yellow (~(1,1)), red (far)
cmap = plt.cm.RdYlGn
colors = [cmap(d) for d in normalized]

# Add lines with gradient coloring
for (label_str, tfp_vals), rgba in zip(results_new.items(), colors):
    color_hex = f'rgb({int(rgba[0]*255)}, {int(rgba[1]*255)}, {int(rgba[2]*255)})'
    fig.add_trace(
        go.Scatter(
            x=years,
            y=tfp_vals,
            mode='lines',
            name=label_str,
            line=dict(color=color_hex),
            hovertemplate='TFP=%{y:.4f}<br>' + label_str
        )
    )


# Update chart layout
fig.update_layout(
    title = 'Acemoglu vs. Overconfidence/Asymmetry Variations',
    xaxis_title = 'Year',
    yaxis_title = 'TFP',
    hovermode = 'closest'
)


<ipython-input-86-bff677ce9327>:9: RuntimeWarning:

overflow encountered in exp

<ipython-input-86-bff677ce9327>:74: RuntimeWarning:

overflow encountered in scalar multiply

<ipython-input-86-bff677ce9327>:77: RuntimeWarning:

overflow encountered in scalar multiply



In [ ]:

# 1) Build data for TFP, as in your snippet.
#    We'll parse out Overconfidence from the scenario label.

mean_tfp_dict = {}

# --- (a) Acemoglu scenario
mean_tfp_acemoglu = np.mean(results_ace)  # average TFP over 20 years
## ADD ACEMOGLU ## mean_tfp_dict["Acemoglu"] = mean_tfp_acemoglu

# --- (b) New approach scenarios
# results_new is a dict: { "OC=0.3,IA=0.1": array_of_TFP_over_20_years, ... }
for label_str, tfp_vals in results_new.items():
    mean_tfp = np.mean(tfp_vals)
    mean_tfp_dict[label_str] = mean_tfp

scenarios = []
mean_tfps = []
overconf_values = []  # This will store numeric overconfidence

for scenario_label, tfp_val in mean_tfp_dict.items():
    scenarios.append(scenario_label)
    mean_tfps.append(tfp_val)

    # Default OC=0 if we can't parse
    oc_float = 0.0

    # Check if label includes something like "OC=0.3"
    if scenario_label.startswith("OC="):
        # Example label: "OC=0.3,IA=0.1"
        parts = scenario_label.split(",")  # => ["OC=0.3", "IA=0.1"]
        oc_str = parts[0].split("=")[1]    # => "0.3"
        oc_float = float(oc_str)

    overconf_values.append(oc_float)

df_bar = pd.DataFrame({
    "Scenario": scenarios,
    "MeanTFP": mean_tfps,
    "Overconfidence": overconf_values
})

# 2) Create a bar chart with color="Overconfidence"
#    We'll pick a single-hue color scale for a gradient effect
fig_bar = px.bar(
    df_bar,
    x="Scenario",
    y="MeanTFP",
    text="MeanTFP",
    color="Overconfidence",
    color_continuous_scale=px.colors.sequential.Blues,  # single-hue range from light->dark
    title="Aggregate TFP Over Setup Years Years (Colored by Overconfidence Level)"
)

# Customize hover info, rotate labels, and tweak layout
fig_bar.update_traces(
    hovertemplate="<b>%{x}</b><br>Mean TFP=%{y:.2f}<br>OC=%{marker.color:.2f}",
    texttemplate='%{text:.2f}'
)
fig_bar.update_layout(
    xaxis_title="Approach / Scenario",
    yaxis_title="Mean TFP (YoY average)",
)
fig_bar.update_xaxes(tickangle=45)


## Viz

In [ ]:
fig_hist.show()
fig.show()
fig_bar.show()

## Trended TFP and By Scenario (DONE)
task_complexities = np.random.normal(loc=3.2, scale=0.3, size=N)
labor_productivities = np.random.normal(loc=3.2, scale=0.5, size=N) #np.clip(labor_productivities_raw, 0.1, 5.0)
ai_productivities = np.random.normal(loc=3.2, scale=0.3, size=N)

In [ ]:
fig_hist.show()
fig.show()
fig_bar.show()

## run 1 (DONE)
task_complexities = np.random.normal(loc=3.2, scale=0.3, size=N)
labor_productivities = np.random.normal(loc=3.5, scale=0.3, size=N) #np.clip(labor_productivities_raw, 0.1, 5.0)
ai_productivities = np.random.normal(loc=3.2, scale=0.3, size=N)

In [ ]:
fig_hist.show()
fig.show()
fig_bar.show()

## run 2 (in progress)
task_complexities = np.random.normal(loc=3.2, scale=0.3, size=N)
labor_productivities = np.random.normal(loc=2.9, scale=0.3, size=N) #np.clip(labor_productivities_raw, 0.1, 5.0)
ai_productivities = np.random.normal(loc=3.2, scale=0.3, size=N)

In [ ]:
fig_hist.show()
fig.show()
fig_bar.show()

## (Do not use)

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
# 1) Build data for TFP, as in your snippet.
#    We'll parse out Overconfidence from the scenario label.
scenarios = []
mean_tfps = []
overconf_values = []  # This will store numeric overconfidence

for scenario_label, tfp_val in mean_tfp_dict.items():
    scenarios.append(scenario_label)
    mean_tfps.append(tfp_val)

    # Default OC=0 if we can't parse
    oc_float = 0.0

    # Check if label includes something like "OC=0.3"
    if scenario_label.startswith("OC="):
        # Example label: "OC=0.3,IA=0.1"
        parts = scenario_label.split(",")  # => ["OC=0.3", "IA=0.1"]
        oc_str = parts[0].split("=")[1]    # => "0.3"
        oc_float = float(oc_str)

    overconf_values.append(oc_float)

df_bar = pd.DataFrame({
    "Scenario": scenarios,
    "MeanTFP": mean_tfps,
    "Overconfidence": overconf_values
})

# 2) Create a bar chart with color="Overconfidence"
#    We'll pick a single-hue color scale for a gradient effect
fig_bar = px.bar(
    df_bar,
    x="Scenario",
    y="MeanTFP",
    text="MeanTFP",
    color="Overconfidence",
    color_continuous_scale=px.colors.sequential.Blues,  # single-hue range from light->dark
    title="Aggregate TFP Over Setup Years Years (Colored by Overconfidence Level)"
)

# Customize hover info, rotate labels, and tweak layout
fig_bar.update_traces(
    hovertemplate="<b>%{x}</b><br>Mean TFP=%{y:.2f}<br>OC=%{marker.color:.2f}",
    texttemplate='%{text:.2f}'
)
fig_bar.update_layout(
    xaxis_title="Approach / Scenario",
    yaxis_title="Mean TFP (20-year average)",
)
fig_bar.update_xaxes(tickangle=45)


NameError: name 'mean_tfp_dict' is not defined

## work in progress (Do not use)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1) COMPUTE AGGREGATE (MEAN) TFP FOR EACH SCENARIO
mean_tfp_dict = {}

# --- (a) Acemoglu scenario
mean_tfp_acemoglu = np.mean(results_ace)  # average TFP over 20 years
mean_tfp_dict["Acemoglu"] = mean_tfp_acemoglu

# --- (b) New approach scenarios
# results_new is a dict: { "OC=0.3,IA=0.1": array_of_TFP_over_20_years, ... }
for label_str, tfp_vals in results_new.items():
    mean_tfp = np.mean(tfp_vals)
    mean_tfp_dict[label_str] = mean_tfp

# ---------------------------------------------------
# 2) BAR CHART: Each bar = aggregated TFP over 20 yrs
# ---------------------------------------------------
fig1 = plt.figure()
approaches = list(mean_tfp_dict.keys())
mean_tfps  = list(mean_tfp_dict.values())

scenarios = []
mean_tfps = []
for scenario_label, tfp_val in mean_tfp_dict.items():
    scenarios.append(scenario_label)
    mean_tfps.append(tfp_val)

df_bar = pd.DataFrame({
    "Scenario": scenarios,
    "MeanTFP": mean_tfps
})

fig_bar = px.bar(
    df_bar,
    x="Scenario",
    y="MeanTFP",
    text="MeanTFP",   # display TFP value as text on bars
    title="Aggregate TFP Over 20 Years (Interactive Bar Chart)"
)

# Customize hover info, rotate labels, and tweak layout:
fig_bar.update_traces(hovertemplate="<b>%{x}</b><br>Mean TFP=%{y:.4f}", texttemplate='%{text:.2f}')
fig_bar.update_layout(
    xaxis_title="Approach / Scenario",
    yaxis_title="Mean TFP (20-year average)",
)
fig_bar.update_xaxes(tickangle=45)  # tilt scenario labels if they're long
fig_bar.show()

<Figure size 640x480 with 0 Axes>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

# 1) Build data for TFP, as in your snippet.
#    We'll parse out Overconfidence from the scenario label.
scenarios = []
mean_tfps = []
overconf_values = []  # This will store numeric overconfidence

for scenario_label, tfp_val in mean_tfp_dict.items():
    scenarios.append(scenario_label)
    mean_tfps.append(tfp_val)

    # Default OC=0 if we can't parse
    oc_float = 0.0

    # Check if label includes something like "OC=0.3"
    if scenario_label.startswith("OC="):
        # Example label: "OC=0.3,IA=0.1"
        parts = scenario_label.split(",")  # => ["OC=0.3", "IA=0.1"]
        oc_str = parts[0].split("=")[1]    # => "0.3"
        oc_float = float(oc_str)

    overconf_values.append(oc_float)

df_bar = pd.DataFrame({
    "Scenario": scenarios,
    "MeanTFP": mean_tfps,
    "Overconfidence": overconf_values
})

# 2) Create a bar chart with color="Overconfidence"
#    We'll pick a single-hue color scale for a gradient effect
fig_bar = px.bar(
    df_bar,
    x="Scenario",
    y="MeanTFP",
    text="MeanTFP",
    color="Overconfidence",
    color_continuous_scale=px.colors.sequential.Blues,  # single-hue range from light->dark
    title="Aggregate TFP Over 20 Years (Colored by Overconfidence Level)"
)

# Customize hover info, rotate labels, and tweak layout
fig_bar.update_traces(
    hovertemplate="<b>%{x}</b><br>Mean TFP=%{y:.2f}<br>OC=%{marker.color:.2f}",
    texttemplate='%{text:.2f}'
)
fig_bar.update_layout(
    xaxis_title="Approach / Scenario",
    yaxis_title="Mean TFP (20-year average)",
)
fig_bar.update_xaxes(tickangle=45)
fig_bar.show()


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

# We assume mean_tfp_dict = { "OC=0.3,IA=0.1": 2.05, "OC=0.5,IA=0.2": 2.10, ... } etc.

scenarios_ia = []
mean_tfps = []
overconf_values = []
info_asym_values = []

for scenario_label, tfp_val in mean_tfp_dict.items():
    # Default oc, ia if we can't parse them:
    oc_float = 0.0
    ia_float = 0.0

    # Attempt to parse strings like "OC=0.3,IA=0.1"
    if scenario_label.startswith("OC="):
        parts = scenario_label.split(",")  # ["OC=0.3", "IA=0.1"]
        # Overconfidence
        oc_str = parts[0].split("=")[1]  # "0.3"
        oc_float = float(oc_str)
        # Info asymmetry
        if len(parts) > 1 and "IA=" in parts[1]:
            ia_str = parts[1].split("=")[1]  # "0.1"
            ia_float = float(ia_str)

    mean_tfps.append(tfp_val)
    overconf_values.append(oc_float)
    info_asym_values.append(ia_float)

    # For the x-axis label, we'll just keep the IA value:
    # e.g. "IA=0.1"
    label_ia = f"IA={ia_float:.2f}"
    scenarios_ia.append(label_ia)

df_bar = pd.DataFrame({
    "ScenarioIA": scenarios_ia,
    "MeanTFP": mean_tfps,
    "Overconfidence": overconf_values,
    "InfoAsym": info_asym_values
})

fig_bar = px.bar(
    df_bar,
    x="ScenarioIA",          # only show "IA=..." on x-axis
    y="MeanTFP",
    text="MeanTFP",
    color="Overconfidence",  # or color="Overconfidence" if you like
    color_continuous_scale=px.colors.sequential.Blues,
    title="Aggregate TFP (X-axis = IA Values Only)"
)

# Customize hover info, rotate labels, and layout
fig_bar.update_traces(
    hovertemplate="<b>%{x}</b><br>Mean TFP=%{y:.2f}"
)
fig_bar.update_layout(
    xaxis_title="Information Asymmetry (IA)",
    yaxis_title="Mean TFP (20-year avg)",
)
fig_bar.update_xaxes(tickangle=45)
fig_bar.show()
